# En este código crearemos las bases de datos y insertaremos los documentos  en su respectiva coleccion de mongoDB realizando los tratamientos necesarios

In [2]:
pip install pymongo

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
import json
import os
from pymongo import MongoClient

## Primero comprobaremos tanto que la base de datos y las colecciones existan, en caso de que no las crearemos

In [5]:
client = MongoClient('mongodb://localhost:27017/')
# Nombre de la BD
db = client["TFM_MTG"]
# Nombres de las colecciones
collection_names = ["AllCards", "AllDecks"]
#Colecciones existentes en la basde de datos
collections = db.list_collection_names()

In [6]:
# Usaremos este bucle para comprobar si las colecciones existen y crearlass en caso negativo
for collection in collection_names:
    if collection not in collections:
        db.create_collection(collection)
        print(f"La colección '{collection}' ha sido creada en la base de datos.")

In [7]:
#Mediante este bloque dividiremos el json para poder importarlo a una base de datos de mongoDB
def split_json(file_path, output_prefix, max_docs_per_file):
    with open(file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
        
        keys = list(data.keys())
        num_files = (len(keys) // max_docs_per_file) + (1 if len(keys) % max_docs_per_file != 0 else 0)
        
        for i in range(num_files):
            start_index = i * max_docs_per_file
            end_index = start_index + max_docs_per_file
            chunk_keys = keys[start_index:end_index]
            
            chunk = {key: data[key] for key in chunk_keys}
            
            output_file_path = f"{output_prefix}_{i+1}.json"
            with open(output_file_path, 'w', encoding='utf-8') as output_file:
                json.dump(chunk, output_file, ensure_ascii=False, indent=4)
            print(f"Escrito {output_file_path}")

In [8]:
input_file = '../Datos/AllCards.json'
output_prefix = '../Datos/Divisiones/AllCards_part'
max_docs_per_file = 2000

split_json(input_file, output_prefix, max_docs_per_file)

Escrito ../Datos/Divisiones/AllCards_part_1.json
Escrito ../Datos/Divisiones/AllCards_part_2.json
Escrito ../Datos/Divisiones/AllCards_part_3.json
Escrito ../Datos/Divisiones/AllCards_part_4.json
Escrito ../Datos/Divisiones/AllCards_part_5.json
Escrito ../Datos/Divisiones/AllCards_part_6.json
Escrito ../Datos/Divisiones/AllCards_part_7.json
Escrito ../Datos/Divisiones/AllCards_part_8.json
Escrito ../Datos/Divisiones/AllCards_part_9.json
Escrito ../Datos/Divisiones/AllCards_part_10.json
Escrito ../Datos/Divisiones/AllCards_part_11.json


## Crearemos la conexion con la bd de mongo para insertar los datos

In [10]:
# Definición de la conexion con mongo
collection = db['AllCards']

In [11]:
num_documents = collection.count_documents({})
print(f"Número de documentos en la colección: {num_documents}")

Número de documentos en la colección: 0


In [12]:
# Definimos la ruta de las divisiones y contamos el numero de ficheros
path_dv ='../Datos/Divisiones/'
fileNumber = [f for f in os.listdir(path_dv) if f.endswith('.json')]

In [13]:
i = 1
for filename in fileNumber:
    fileJson = f'AllCards_part_{i}.json'
    filepath = os.path.join(path_dv, fileJson)
    try:
        # Abrir el archivo JSON y leer su contenido
        with open(filepath, 'r', encoding='utf-8') as file:
            json_data = json.load(file)

        # Insertar los documentos en la colección de MongoDB
        result = collection.insert_many(json_data.values())
        print(f"{fileJson}: {len(result.inserted_ids)} documentos insertados correctamente.")
        
    except FileNotFoundError:
        print(f"El archivo {fileJson} no se encontró.")
    except json.JSONDecodeError as e:
        print(f"Error al decodificar el archivo {fileJson}: {e}")
    except Exception as e:
        print(f"Error al insertar el archivo {fileJson} en la base de datos: {e}")
        
    i += 1


AllCards_part_1.json: 2000 documentos insertados correctamente.
AllCards_part_2.json: 2000 documentos insertados correctamente.
AllCards_part_3.json: 2000 documentos insertados correctamente.
AllCards_part_4.json: 2000 documentos insertados correctamente.
AllCards_part_5.json: 2000 documentos insertados correctamente.
AllCards_part_6.json: 2000 documentos insertados correctamente.
AllCards_part_7.json: 2000 documentos insertados correctamente.
AllCards_part_8.json: 2000 documentos insertados correctamente.
AllCards_part_9.json: 2000 documentos insertados correctamente.
AllCards_part_10.json: 2000 documentos insertados correctamente.
AllCards_part_11.json: 478 documentos insertados correctamente.


## Realizaremos un proceso similar para almacenar en mongo los datos de los mazos

In [15]:
#Selecionaremos la coleccion donde almacenaremos los mazos y modificaremos la ruta donde estan los JSON
collection = db['AllDecks']

path_dv = "..\Datos\AllDeckFiles"
fileNumber = [f for f in os.listdir(path_dv) if f.endswith('.json')]

In [16]:
for filename in fileNumber:
    file_path = os.path.join(path_dv, filename)
    with open(file_path, 'r', encoding='utf-8') as file:
        try:
            data = json.load(file)
            collection.insert_one(data)
            print(f"Mazo {filename} insertado.")
        except json.JSONDecodeError as e:
            print(f"Error al decodificar el archivo {filename}: {e}")
        except Exception as e:
            print(f"Error al insertar el archivo {filename} en la base de datos: {e}")

Mazo AbzanSiege_KTK.json insertado.
Mazo AdaptiveEnchantment_C18.json insertado.
Mazo Aerodoom_BTD.json insertado.
Mazo AirForces_WTH.json insertado.
Mazo AirRazers_JUD.json insertado.
Mazo AjaniGoldmane_DDH.json insertado.
Mazo AjaniInspiringLeader_M20.json insertado.
Mazo AjaniValiantProtector_AER.json insertado.
Mazo Ajani_M19.json insertado.
Mazo AmonkhetWelcomeDeckBlack_W17.json insertado.
Mazo AmonkhetWelcomeDeckBlue_W17.json insertado.
Mazo AmonkhetWelcomeDeckGreen_W17.json insertado.
Mazo AmonkhetWelcomeDeckRed_W17.json insertado.
Mazo AmonkhetWelcomeDeckWhite_W17.json insertado.
Mazo AngelicFury_SOI.json insertado.
Mazo AngelicMight_AVR.json insertado.
Mazo AngrathMinotaurPirate_RIX.json insertado.
Mazo AnthousaSArmy_THS.json insertado.
Mazo ArcaneTempo_GRN.json insertado.
Mazo ArcaneWizardry_C17.json insertado.
Mazo ArcanisSGuile_10E.json insertado.
Mazo ArchenemyNicolBolas_E01.json insertado.
Mazo Armada_7ED.json insertado.
Mazo ArmedAndDangerous_ORI.json insertado.
Mazo Arm